In [1]:
import torch
from diffusers import StableDiffusionPipeline

2025-04-04 11:56:19.817935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743760579.843326  753197 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743760579.856175  753197 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 11:56:19.887619: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Load the Stable Diffusion model

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

h:\GIT_win\GENAI PROJET\genai_project\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\CYTech Student\.cache\huggingface\hub\models--CompVis--stable-diffusion-v1-4. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Get text embeddings

In [ ]:
max_length = 50

tokenizer = pipe.tokenizer
text_encoder = pipe.text_encoder

prompt = "A futuristic car"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

text_encoder = text_encoder
diffusion_model = pipe.unet
timesteps = 100
criterion = nn.MSELoss()
optimizer = optim.Adam(diffusion_model.parameters(), lr=1e-4)

In [ ]:
def optimize(target_text, num_steps=5):
    tokens = tokenizer(target_text, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    with torch.no_grad():
        e_tgt = text_encoder(tokens)

    # freeze diffusion model parameters
    for param in diffusion_model.parameters():
        param.requires_grad = False

    x = e_tgt.clone().detach()
    for _ in range(num_steps):
        t = torch.randint(1, timesteps + 1, (1,))
        epsilon = torch.randn_like(x) #noise
        x_t = x + epsilon
        
        e_pred = diffusion_model(x_t, t, e_tgt)

        loss = criterion(e_pred, e_tgt)

        # update embeddings
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    for param in diffusion_model.parameters():
        param.requires_grad = True

    # get the final embedding after update
    with torch.no_grad():
        x = text_encoder(tokens)

    return x

optimize(prompt)
